In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# 
plt.rcParams.update({
    'font.family': 'Arial',
    'font.size': 12,
    'axes.linewidth': 1.2,
    'xtick.major.size': 6,
    'xtick.major.width': 1.2,
    'ytick.major.size': 6,
    'ytick.major.width': 1.2,
    'axes.edgecolor': 'black',
    'figure.dpi': 1200,
    'savefig.dpi': 1200
})

# File paths to save the plots
save_path_pdf = "S2.pdf"

# Load the evaluation metrics CSV file
eval_df = pd.read_csv("evaluation_metrics.csv") #The same file used here which we obtained from the fine tuning of the final model (Figure 1C)

# Feature name processing
eval_df['feature_set'] = (
    eval_df['feature_set']
    .str.replace(r'\+', '&', regex=True)
    .str.replace(r'\s*&\s*', ' & ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

def count_features(name: str) -> int:
    """Calculate feature complexity by counting the number of features separated by '&'."""
    parts = [part.strip() for part in name.split('&')]
    return min(len(parts), 4)

# Compute feature complexity
eval_df['complexity'] = eval_df['feature_set'].apply(count_features)

# Calculate statistics for both metrics
def prepare_stats(metric, sort_by_spearman=False):
    stats = (
        eval_df
        .groupby('feature_set')[metric]
        .agg(['mean', 'std'])
        .reset_index()
    )
    stats['complexity'] = stats['feature_set'].apply(count_features)
    stats.loc[stats['feature_set'].str.lower() == 'all features', 'complexity'] = 4
    
    # If we want to sort by Spearman ranking (for MSE plot)
    if sort_by_spearman:
        spearman_means = eval_df.groupby('feature_set')['spearman'].mean()
        stats = stats.set_index('feature_set').join(spearman_means.rename('spearman_mean'))
        stats = stats.sort_values('spearman_mean', ascending=True).reset_index()
    
    return stats

# Prepare stats - sort Spearman normally, but MSE by Spearman ranking
spearman_stats = prepare_stats('spearman').sort_values('mean', ascending=True)
mse_stats = prepare_stats('mse', sort_by_spearman=True)

# Define colors corresponding to feature complexity levels
feature_colors = {
    1: '#4E79A7',  # Blue
    2: '#F28E2B',  # Orange
    3: '#E15759',  # Red
    4: '#76B7B2'   # Teal
}

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), dpi=1200, sharey=True)
y_pos = np.arange(len(spearman_stats))

# Plot Spearman correlation
colors = spearman_stats['complexity'].map(feature_colors)
bars1 = ax1.barh(y_pos, spearman_stats['mean'], color=colors, edgecolor='black', height=0.7)
ax1.errorbar(spearman_stats['mean'], y_pos, xerr=spearman_stats['std'],
            fmt='none', ecolor='black', elinewidth=1.5, capsize=5)

# Plot individual points for Spearman
for i, feature_set in enumerate(spearman_stats['feature_set']):
    vals = eval_df[eval_df['feature_set'] == feature_set]['spearman'].values
    ax1.scatter(vals, np.full_like(vals, y_pos[i]), color='black', s=30, alpha=0.5, edgecolor='none')

# Configure Spearman x-axis
ax1.set_xlim(0.6, 0.85)
ax1.set_xticks(np.arange(0.6, 0.86, 0.05))
ax1.set_xlabel('Spearman correlation', fontsize=12)

# Plot MSE (using same order as Spearman plot)
colors = mse_stats['complexity'].map(feature_colors)
bars2 = ax2.barh(y_pos, mse_stats['mean'], color=colors, edgecolor='black', height=0.7)
ax2.errorbar(mse_stats['mean'], y_pos, xerr=mse_stats['std'],
            fmt='none', ecolor='black', elinewidth=1.5, capsize=5)

# Plot individual points for MSE
for i, feature_set in enumerate(mse_stats['feature_set']):
    vals = eval_df[eval_df['feature_set'] == feature_set]['mse'].values
    ax2.scatter(vals, np.full_like(vals, y_pos[i]), color='black', s=30, alpha=0.5, edgecolor='none')

# Configure MSE x-axis
ax2.set_xlim(0.2, 0.6)
ax2.set_xticks(np.arange(0.2, 0.61, 0.1))
ax2.set_xlabel('Mean Squared Error (MSE)', fontsize=12)

# Configure shared y-axis (using Spearman order)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(spearman_stats['feature_set'], fontsize=12)

# Add grid and remove ticks
for ax in [ax1, ax2]:
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(False)

# Create a single legend for both plots
legend_handles = [
    Patch(color=color, label=f'{count} feature{"s" if count > 1 else ""}')
    for count, color in feature_colors.items()
]
fig.legend(handles=legend_handles, title='Feature complexity',
          fontsize=12, title_fontsize=12,
          loc='center right', frameon=False)

# Adjust layout to leave space for the legend
plt.tight_layout(rect=[0, 0, 0.9, 1])

# Save the plot in multiple formats
plt.savefig(save_path_tiff, dpi=1200, bbox_inches='tight')
plt.savefig(save_path_svg, dpi=1200, bbox_inches='tight')
plt.savefig(save_path_pdf, dpi=1200, bbox_inches='tight')

plt.show()
print(f"Plot saved to {save_path_tiff}, {save_path_svg}, and {save_path_pdf}")

Plot saved to /mnt/e/Final_files/combined_metrics_comparison.tiff, /mnt/e/Final_files/combined_metrics_comparison.svg, and /mnt/e/Final_files/combined_metrics_comparison.pdf
